In [29]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
inbound = pd.read_csv('../Data/Inbound.csv')
outbound = pd.read_csv('../Data/outbound.csv')
inventory = pd.read_csv('../Data/inventory.csv')
operation = pd.read_csv('../Data/operationCost.csv')
material = pd.read_csv('../Data/materialMaster.csv')

In [31]:
# inboundSG = inbound[inbound['PLANT_NAME'] == 'SINGAPORE-WAREHOUSE']
# inboundSG.groupby(['INBOUND_DATE', 'MATERIAL_NAME']).agg(
#     total_quantity=('NET_QUANTITY_MT', 'sum')
# ).reset_index().sort_values(by='INBOUND_DATE')

In [32]:
# inboundChina = inbound[inbound['PLANT_NAME'] == 'CHINA-WAREHOUSE']
# inboundChina.groupby(['INBOUND_DATE', 'MATERIAL_NAME']).agg(
#     total_quantity=('NET_QUANTITY_MT', 'sum')
# ).reset_index().sort_values(by='INBOUND_DATE')

In [58]:
inventory.head(10)

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
0,12/31/2023,CHINA-WAREHOUSE,MAT-0045,SCRAP,164,KG,211,CNY
1,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6024,0,KG,87666,CNY
2,12/31/2023,CHINA-WAREHOUSE,MAT-0193,5755,70720,KG,414419,CNY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY
4,12/31/2023,CHINA-WAREHOUSE,MAT-0193,6734,2720,KG,15939,CNY
5,12/31/2023,CHINA-WAREHOUSE,MAT-0311,SCRAP,4142,KG,25018,CNY
6,12/31/2023,CHINA-WAREHOUSE,MAT-0374,5995C5416,1000,KG,6430,CNY
7,12/31/2023,CHINA-WAREHOUSE,MAT-0374,8466C2971,15125,KG,97254,CNY
8,12/31/2023,CHINA-WAREHOUSE,MAT-0374,8143C2373,32975,KG,212029,CNY
9,12/31/2023,CHINA-WAREHOUSE,MAT-0046,9532,15125,KG,97254,CNY


In [57]:
inventory[inventory['BATCH_NUMBER'] == '3142']

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
3,12/31/2023,CHINA-WAREHOUSE,MAT-0193,3142,12240,KG,71726,CNY


In [10]:
inventory_grouped = inventory.groupby(['MATERIAL_NAME']).agg(
    total_quantity=('UNRESRICTED_STOCK', 'sum'),
    total_cost=('STOCK_SELL_VALUE', 'sum')
).reset_index().sort_values(by='total_quantity', ascending=True)

inventory_grouped[inventory_grouped['MATERIAL_NAME'] == 'MAT-0045']

,MATERIAL_NAME,total_quantity,total_cost
44,MAT-0045,2826,3097


In [17]:
inventory[inventory['MATERIAL_NAME'] == 'MAT-0331']

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,BATCH_NUMBER,UNRESRICTED_STOCK,STOCK_UNIT,STOCK_SELL_VALUE,CURRENCY
639,12/31/2023,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,70,CNY
1454,1/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
2314,2/29/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
3213,3/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
4134,4/30/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
5080,5/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
6278,6/30/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
7924,7/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,0,CNY
9387,8/31/2024,CHINA-WAREHOUSE,MAT-0331,SCRAP,12,KG,70,CNY


In [40]:
# Scraps
inventory_scrap = inventory[inventory['BATCH_NUMBER'] == 'SCRAP']


inventory_scrap_merged = inventory_scrap.merge(
    material[['MATERIAL_NAME', 'SHELF_LIFE_IN_MONTH']],
    on='MATERIAL_NAME',
    how='left'
).drop(columns=['STOCK_UNIT', 'CURRENCY', 'BATCH_NUMBER'])

inventory_scrap_merged.sort_values(by='BALANCE_AS_OF_DATE')

,BALANCE_AS_OF_DATE,PLANT_NAME,MATERIAL_NAME,UNRESRICTED_STOCK,STOCK_SELL_VALUE,SHELF_LIFE_IN_MONTH
32,1/31/2024,CHINA-WAREHOUSE,MAT-0279,1654,3424,3
22,1/31/2024,CHINA-WAREHOUSE,MAT-0257,5164,10482,8
23,1/31/2024,CHINA-WAREHOUSE,MAT-0258,11187,65558,9
24,1/31/2024,CHINA-WAREHOUSE,MAT-0100,5143,31063,5
25,1/31/2024,CHINA-WAREHOUSE,MAT-0095,1685,2325,9
...,...,...,...,...,...,...
208,9/30/2024,CHINA-WAREHOUSE,MAT-0258,529,3098,9
206,9/30/2024,CHINA-WAREHOUSE,MAT-0194,38,118,2
205,9/30/2024,CHINA-WAREHOUSE,MAT-0045,68,88,10
210,9/30/2024,CHINA-WAREHOUSE,MAT-0095,157,217,9


In [38]:
temp = inventory_scrap_merged.groupby(['MATERIAL_NAME']).agg(
    total_quantity=('UNRESRICTED_STOCK', 'sum'),
    total_cost=('STOCK_SELL_VALUE', 'sum'),
    shelf_life=('SHELF_LIFE_IN_MONTH', 'first'),
).reset_index().sort_values(by='total_quantity', ascending=True)

temp.head()

,MATERIAL_NAME,total_quantity,total_cost,shelf_life
17,MAT-0219,20,11,6
27,MAT-0331,108,140,6
2,MAT-0036,130,726,5
6,MAT-0078,152,512,3
0,MAT-0010,495,3172,5


In [52]:
inbound[inbound['MATERIAL_NAME'] == 'MAT-0010']

,INBOUND_DATE,PLANT_NAME,MATERIAL_NAME,NET_QUANTITY_MT


In [9]:
material.head()

,MATERIAL_NAME,POLYMER_TYPE,SHELF_LIFE_IN_MONTH,DOWNGRADE_VALUE_LOST_PERCENT
0,MAT-0001,P-002,3,40
1,MAT-0002,P-001,8,15
2,MAT-0003,P-004,2,35
3,MAT-0004,P-002,3,35
4,MAT-0005,P-002,5,20


In [ ]:
today = pd.to_datetime('2025-06-30')

Timestamp('2025-06-30 00:00:00')